In [8]:
import gym
import numpy as np
import tqdm

In [2]:
env = gym.make('FrozenLake-v1')

In [ ]:
env.reset()

In [4]:
n_obs = env.observation_space.n
n_act = env.action_space.n

In [26]:
returns_sum = np.zeros(n_obs)
returns_count = np.zeros(n_obs)
max_episodes = 10000
max_iters = 1000
gamma = 1.0

In [31]:
def sample_policy(s):
    d = [0, 3, 3, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0.]
    return d[s]

In [ ]:
for i in tqdm.tqdm(range(max_episodes)):
    s = env.reset()
    episode = []
    for i in range(max_iters):
        a = sample_policy(s)
        next_s, r, done, _ = env.step(a)
        episode.append((s, a, r))
        if done:
            break
        s = next_s

    states_in_episode = set([i[0] for i in episode])
    episode = np.array(episode)
    for s in states_in_episode:
        first_idx = np.where(episode[:, 0]==s)[0][0]
        G = sum([r * (gamma ** i) for i, (s, a, r) in enumerate(episode[first_idx:])])
        returns_sum[s] += G
        returns_count[s] += 1

v_s = returns_sum / (returns_count+1e-10)

In [11]:
def value_iteration(env, gamma = 1.0):
    
    value_table = np.zeros(env.observation_space.n)
    
    no_of_iterations = 100000
    threshold = 1e-20
    
    for i in range(no_of_iterations):
        
        updated_value_table = np.copy(value_table) 
        
        for state in range(env.observation_space.n):
            Q_value = []
            for action in range(env.action_space.n):
                next_states_rewards = []
                for next_sr in env.P[state][action]: 
                    trans_prob, next_state, reward_prob, _ = next_sr 
                    next_states_rewards.append((trans_prob * (reward_prob + gamma * updated_value_table[next_state]))) 
                
                Q_value.append(np.sum(next_states_rewards))
                
            value_table[state] = max(Q_value) 
            
        
        if (np.sum(np.fabs(updated_value_table - value_table)) <= threshold):
            print('Value-iteration converged at iteration# %d.' %(i+1))
            break
    
    return value_table

In [ ]:
v = value_iteration(env, gamma=1.0)